# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(5.5))
x

[101752172138391560107742343723197438423,
 112643174959466551314976851212464508707,
 79577998377309135647726082845253797283]

In [3]:
y = encrypt(encode(2.3))
y

[65888619123161317341439054298205254529,
 85213716932118374307349271496161235920,
 142871009419887555421656951986229253963]

In [4]:
z = add(x,y)
z

(167640791261552877449181398021402692952,
 197856891891584925622326122708625744627,
 222449007797196691069383034831483051246)

In [5]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[37614788550239505264997883166877892331,
 247784717455248558664230516988114572113,
 8573839469679183141216877625923279969]

In [22]:
z = imul(x, 2)

In [23]:
decode(decrypt(z))

11.0

# Lesson: Encrypted Computation in PySyft

In [26]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0722 22:36:23.252707 4321358720 hook.py:98] Torch was already hooked... skipping hooking process


In [27]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [28]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [29]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [30]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [31]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [32]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [33]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [35]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [37]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [39]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [40]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [41]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [42]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [43]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [44]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [45]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [46]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [47]:
# try this project here!
import string
import torch as th

char2index = {}
index2char = {}

for i, char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [48]:
str_input = 'Hello'

In [49]:
def string2values(str_input, max_len=8):
    str_input = str_input[:max_len].lower()
    
    if len(str_input) < max_len:
        str_input = str_input.ljust(max_len, '.')
        
    values = []
    for ch in str_input:
        values.append(char2index[ch])

    return th.tensor(values)

def values2string(input_values):
    s = ''
    for value in input_values:
        s += index2char[int(value)]
    
    return s

In [50]:
string2values(str_input)

tensor([ 8,  5, 12, 12, 15, 50, 50, 50])

In [51]:
def one_hot(index, length):
    vect = th.zeros(length)
    vect[index] = 1
    
    return vect

In [52]:

one_hot(char2index['p'], len(index2char))


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [53]:
def string2one_hot_matrix(str_input, max_len=8):
    str_input = str_input[:max_len].lower()
    
    if len(str_input) < max_len:
        str_input = str_input.ljust(max_len, '.')
        
    ch_vectors = []
    for ch in str_input:
        ch_v =  one_hot(char2index[ch], len(index2char)).unsqueeze(0)
        ch_vectors.append(ch_v)

    return th.cat(ch_vectors, dim=0)

In [54]:

def string_equal(str_a, str_b):
    vect = (str_a * str_b).sum(1)
    
    str_match = vect[0]
    
    for i in range(1, vect.shape[0]):
        str_match =  str_match * vect[i]
    
    return str_match

In [55]:
str_a = string2one_hot_matrix('Helloooo')
str_b = string2one_hot_matrix('Hello')

(str_a * str_b).sum(1)

tensor([1., 1., 1., 1., 1., 0., 0., 0.])

In [56]:
class EncryptedDB():
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len

        self.keys = []
        self.values = []
        self.owners = owners
    
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value, max_len=self.max_val_len)
        value = value.share(*self.owners)
        self.values.append(value)   

    def query(self, query_str):    
        query_matrix = string2one_hot_matrix(query_str)
        query_matrix = query_matrix.share(*self.owners)
        
        key_matches = []
        for key in self.keys:
            key_match = string_equal(key, query_matrix)
            key_matches.append(key_match)
        
        result = self.values[0] * key_matches[0]
        
        for i in range(1, len(self.values)):
            result += self.values[i] * key_matches[i]
        
        result = result.get()
        
        return values2string(result).replace('.','')


In [58]:
db = EncryptedDB(bob, alice, secure_worker, max_val_len=256)

db.add_entry("Bob","(123) 456 7890")
db.add_entry("Bill", "(234) 567 8901")
db.add_entry("Sam","(345) 678 9012")
db.add_entry("Key","really big json value")

db.query("Bob")


'(123) 456 7890'

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [59]:
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch as th

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.9658)
tensor(19.0660)
tensor(19.1070)
tensor(1.0263)
tensor(0.9510)
tensor(0.9289)
tensor(0.9059)
tensor(0.8793)
tensor(0.8487)
tensor(0.8140)
tensor(0.7756)
tensor(0.7346)
tensor(0.6928)
tensor(0.6523)
tensor(0.6150)
tensor(0.5800)
tensor(0.5038)
tensor(0.4364)
tensor(0.3760)
tensor(0.3222)


In [60]:
model(data)

tensor([[ 0.4369],
        [-0.0014],
        [ 0.9665],
        [ 0.7130]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [61]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [62]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:44979525946 -> alice:75627641649]
 	-> (Wrapper)>[PointerTensor | me:97093027980 -> bob:72979226541]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:45259431729 -> alice:1722802720]
 	-> (Wrapper)>[PointerTensor | me:64613333623 -> bob:3584021719]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:66430493656 -> alice:48244374367]
 	-> (Wrapper)>[PointerTensor | me:63966560061 -> bob:91909883656]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:21966661860 -> alice:18377940984]
 	-> (Wrapper)>[PointerTensor | me:60063525094 -> bob:2131039291

In [63]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [64]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:7832281510 -> alice:60903311354]
	-> (Wrapper)>[PointerTensor | me:14342969204 -> bob:36353382825]
	*crypto provider: secure_worker*

In [65]:
encrypted_prediction = encrypted_model(encrypted_data)

In [66]:
encrypted_prediction.get().float_precision()

tensor([[ 0.4370],
        [-0.0020],
        [ 0.9660],
        [ 0.7120]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [67]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path='mnist.npz')

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0722 22:37:14.705604 4321358720 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Train on 60000 samples, validate on 10000 samples


Epoch 1/2


  128/60000 [..............................] - ETA: 2:45 - loss: 2.2997 - acc: 0.1250

...

59648/60000 [============================>.] - ETA: 0s - loss: 2.2979 - acc: 0.1023

59776/60000 [============================>.] - ETA: 0s - loss: 2.2979 - acc: 0.1022

59904/60000 [============================>.] - ETA: 0s - loss: 2.2979 - acc: 0.1023

60000/60000 [==============================] - 41s 690us/sample - loss: 2.2979 - acc: 0.1023 - val_loss: 2.2967 - val_acc: 0.1015


Test loss: 2.296708573913574
Test accuracy: 0.1015


In [68]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [69]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [70]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [71]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [72]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [73]:
model.share(alice, bob, carol)

W0722 22:39:05.522319 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/keras/engine/base_layer_utils.py:29: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W0722 22:39:05.532347 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W0722 22:39:06.753432 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/tensor/native.py:101: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0722 22:39:06.989641 4321358720 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/tensor/shared.py:62: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a future version.
Instructions for updating:
ksizes is deprecated, use sizes instead


W0722 22:39:07.394921 4321358720 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/protocol/pond/pond.py:3131: The name tf.Dimension is deprecated. Please use tf.compat.v1.Dimension instead.



W0722 22:39:07.573801 4321358720 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tf_encrypted/protocol/securenn/odd_tensor.py:306: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


I0722 22:39:15.083578 4321358720 session.py:55] Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [ ]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [ ]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org